Packages required

In [ ]:
#pip install pandas
#pip install requests

Imports

In [2]:
import pandas as pd
import urllib.request
import json
import os
import shutil

Get all players information

In [3]:
#URL information
url_base = 'https://drop-api.ea.com/rating/fc-24?locale=en'
limit = 100
offset = 0
page = 1

#Json information
file_name = 'players-page'
file_type = '.json'
directory = 'players/'

#Create folder
if not os.path.exists(directory):
    os.mkdir(directory)
else:
    shutil.rmtree(directory)
    os.mkdir(directory)

#Get all player information
while True:
    url = f'{url_base}&limit={limit}&offset={offset}'
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    
    items = data['items']
    
    if len(items) == 0: #or offset == 200:
        break  # Break the loop if there are no more items
    
    urllib.request.urlretrieve(url, f'{directory}{file_name}-{page}{file_type}') # Escribir una nueva línea para separar los items
    
    page += 1
    offset += limit

Create Dataframe

In [4]:
#List to saved all dataframes
dfs = []

#Iterate over all json files
for filename in os.listdir(directory):
        #print(f'Processing file {filename}')
        file_path = os.path.join(directory, filename)
        
        # Cargar el JSON
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # Crear DataFrame
        df = pd.json_normalize(data['items'])
                
        # Agregar el DataFrame a la lista
        dfs.append(df)

#Concatenate all dataframes
final_df = pd.concat(dfs, ignore_index=True)

#Sort it by rank
final_df= final_df.sort_values('rank')

#Count it
final_df.count()
        

id                     17326
rank                   17326
overallRating          17326
firstName              17326
lastName               17326
                       ...  
stats.strength.diff    17326
stats.vision.value     17326
stats.vision.diff      17326
stats.volleys.value    17326
stats.volleys.diff     17326
Length: 112, dtype: int64

Get Best player by position

In [5]:
best_players_idx = final_df.groupby('position.shortLabel')['rank'].idxmin()
best_players = final_df.loc[best_players_idx]

#Select desired columns
best_players = best_players.loc[:, ['rank', 'overallRating', 'firstName', 'lastName', 'commonName', 'birthdate', 'leagueName', 'nationality.label', 'team.label', 'position.shortLabel', 'position.label']]
best_players = best_players.sort_values('rank')

Display best players

In [6]:
from tabulate import tabulate

table_string = tabulate(best_players, headers='keys', tablefmt='pretty')
print(table_string)

+-------+------+---------------+-----------------+--------------------+-----------------+------------------------+--------------------+-------------------+-------------------+---------------------+-----------------------------+
|       | rank | overallRating |    firstName    |      lastName      |   commonName    |       birthdate        |     leagueName     | nationality.label |    team.label     | position.shortLabel |       position.label        |
+-------+------+---------------+-----------------+--------------------+-----------------+------------------------+--------------------+-------------------+-------------------+---------------------+-----------------------------+
| 11726 |  1   |      91       |     Kylian      |       Mbappé       |                 | 12/20/1998 12:00:00 AM | Ligue 1 Uber Eats  |      France       |     Paris SG      |         ST          |           Striker           |
| 11727 |  2   |      91       |     Alexia      |  Putellas Segura   | Alexia Putellas 